In [1]:
import numpy as np
import pandas as pd
import os
from datetime import date
import sqlalchemy
import pymysql
import re
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)

import warnings    # to avoid warning during executions
warnings.filterwarnings("ignore")

In [2]:
file_name = '..\..\..\dataExport\LOGdata\log_2022_4_20.csv'
log_rawDF = pd.read_csv(file_name, sep='*')

In [3]:
log_rawDF.head(5)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
0,2021-4-20 13:0:13,1618916413,613129,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN
1,2021-4-20 13:0:57,1618916457,9327,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN
2,2021-4-20 13:1:38,1618916498,812633,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN
3,2021-4-20 13:2:20,1618916540,214501,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN
4,2021-4-20 13:3:3,1618916583,212528,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN


In [4]:
feed_temp_ins4 = log_rawDF.query('Time >= 1618561800 and Time <= 1625228995 and Unit =="FPC14" and Parameter == "program"')

In [5]:
feed_temp_ins4.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
6,2021-4-20 13:3:58,1618916638,271269,3,FPC14,program,step ID - Changed (new and old value),4_WaterAcid,5_empty
7,2021-4-20 13:4:1,1618916641,101959,3,FPC14,program,start at - Changed (new and old value),sellected step at...,first step
8,2021-4-20 13:4:6,1618916646,386245,3,FPC14,program,procedure running - Changed,not active,active
9,2021-4-20 13:4:7,1618916647,639887,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
10,2021-4-20 13:4:7,1618916647,642814,2,FPC14,program,Step 4_WaterAcid started,NaN,NaN


In [6]:
programID_DF = feed_temp_ins4[feed_temp_ins4['Message'].str.contains('PrgID', regex=False, case=False, na=False)]

In [7]:
programID_DF.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
1436,2021-4-20 22:0:5,1618948805,125813,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
25922,2021-4-21 22:0:5,1619035205,342017,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
27652,2021-4-22 10:0:5,1619078405,339214,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
29425,2021-4-22 22:0:5,1619121605,341318,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
30376,2021-4-23 10:0:5,1619164805,351891,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN


In [8]:
programID_DF['Message'].unique()

array(['program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65',
       'program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 50'],
      dtype=object)

In [9]:
StepID_DF = feed_temp_ins4[feed_temp_ins4['Message'].str.contains('Step', regex=False, case=False, na=False)]

In [10]:
StepID_DF.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
6,2021-4-20 13:3:58,1618916638,271269,3,FPC14,program,step ID - Changed (new and old value),4_WaterAcid,5_empty
10,2021-4-20 13:4:7,1618916647,642814,2,FPC14,program,Step 4_WaterAcid started,NaN,NaN
16,2021-4-20 13:4:25,1618916665,180599,2,FPC14,program,Step 4_WaterAcid started,NaN,NaN
146,2021-4-20 14:4:1,1618920241,169613,2,FPC14,program,Step 5_empty started,NaN,NaN
1438,2021-4-20 22:0:6,1618948806,648291,2,FPC14,program,Step 1_Harvest started,NaN,NaN


In [11]:
StepID_DF['Message'].unique()

array(['step ID - Changed (new and old value)',
       'Step 4_WaterAcid started', 'Step 5_empty started',
       'Step 1_Harvest started', 'Step 2_SiliciumHCl started',
       'Step 3_CompleteNutrients started',
       'Record ProgramID = F_ThalaPseu_HalfNutr  StepID = 1_Harvest saved (old and new record)EmptyReactorHarvestVolumeProductCIPFeedVolumeStopCirculationDuringFeedPneumaticDuringFeedPneumaticTimeDripTimeAerationAfterDripTimeDripCyclesOzoneDuringWaitwaitingTimeHarvestTriggerLastFeedStartedTriggerFeedTriggerStepTrigger',
       'Step 1_water started', 'Step 2_Oxonia 2% started',
       'Step step1 started', 'Step step2 started', 'Step step3 started'],
      dtype=object)

In [12]:
program_DF = feed_temp_ins4[feed_temp_ins4['Message'].str.contains('Program', regex=False, case=False, na=False)]

In [13]:
program_DF.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
9,2021-4-20 13:4:7,1618916647,639887,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
12,2021-4-20 13:4:11,1618916651,125940,2,FPC14,program,_======program F_ThalaPseu_HalfNutr stopped before ended!!===========,NaN,NaN
15,2021-4-20 13:4:25,1618916665,177004,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
172,2021-4-20 14:5:54,1618920354,625413,2,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
1436,2021-4-20 22:0:5,1618948805,125813,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN


In [14]:
L = ['program','Program']
just_programDF = feed_temp_ins4[feed_temp_ins4['Message'].str.startswith(tuple(L))]

In [15]:
just_programDF['Message'].unique()

array(['Program F_ThalaPseu_HalfNutr started',
       'Program F_ThalaPseu_HalfNutr ended',
       'program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65',
       'program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 50',
       'Program CleanOxonia_0.5% started',
       'Program waterSpoeling started', 'Program waterSpoeling ended',
       'Program Clean feed line base started',
       'Program Clean feed line base ended'], dtype=object)

In [16]:
#df[~df.C.str.contains("XYZ")]

only_feed = just_programDF[just_programDF["Message"].str.contains("CleanOxonia|waterSpoeling|line")==False]

In [17]:
only_feed['Message'].unique()

array(['Program F_ThalaPseu_HalfNutr started',
       'Program F_ThalaPseu_HalfNutr ended',
       'program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65',
       'program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 50'],
      dtype=object)

In [18]:
only_feed.head(20)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
9,2021-4-20 13:4:7,1618916647,639887,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
15,2021-4-20 13:4:25,1618916665,177004,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
172,2021-4-20 14:5:54,1618920354,625413,2,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
1436,2021-4-20 22:0:5,1618948805,125813,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
1437,2021-4-20 22:0:6,1618948806,644921,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
1746,2021-4-20 23:40:9,1618954809,644779,2,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
24188,2021-4-21 11:23:12,1618996992,630347,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
24625,2021-4-21 13:26:31,1619004391,861094,2,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
25922,2021-4-21 22:0:5,1619035205,342017,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
25923,2021-4-21 22:0:6,1619035206,860679,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN


In [107]:
'''
(only_feed
 .assign(col=only_feed.groupby('Message').cumcount().add(1))
 .pivot(index='Message', columns='col', values='Time')
 .add_prefix('Time_')
 .reset_index().rename_axis(columns=None)
)
'''

"\n(only_feed\n .assign(col=only_feed.groupby('Message').cumcount().add(1))\n .pivot(index='Message', columns='col', values='Time')\n .add_prefix('Time_')\n .reset_index().rename_axis(columns=None)\n)\n"

In [94]:
#feed_reactor_instance4 = log_rawDF.query('Type == 2 and Parameter == "Reactor" and Time >= 1618561800 and Time <= 1625228995 and Unit =="FPC14"')

In [113]:
# manual_ins4 = log_rawDF.query('Time >= 1618561800 and Time <= 1625228995 and Unit =="FPC14" and Type == 3')

In [125]:
# only_feed
'''
for index, row in only_feed.iterrows():
    start_msg = row['Message']
    words = start_msg.split()
    check = "Vol"
    if check in words:
        new_msg = start_msg
    

for row in only_feed.itertuples():
    print(row.Type)
    print(row.Index)
'''



In [142]:
only_feed_newIndex = only_feed.reset_index()

In [143]:
only_feed_newIndex= only_feed_newIndex.drop('index',1)

In [144]:
only_feed_newIndex.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
0,2021-4-20 13:4:7,1618916647,639887,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
1,2021-4-20 13:4:25,1618916665,177004,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
2,2021-4-20 14:5:54,1618920354,625413,2,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
3,2021-4-20 22:0:5,1618948805,125813,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
4,2021-4-20 22:0:6,1618948806,644921,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN


In [119]:
'''
for idex, row in only_feed_newIndex.iterrows():
    start_msg = row['Message']
    words = start_msg.split()
    check = "Vol"
    if check in words:
        new_msg = start_msg

    check_str = "ended"
    if "ended" in start_msg:
        next_le = only_feed_newIndex.loc[idex+1]['Message']
        if "Vol" not in next_le:
            only_feed_newIndex.loc[idex+0.5] = new_msg
            
only_feed_newIndex = only_feed_newIndex.sort_index().reset_index(drop=True)
'''
    

'\nfor idex, row in only_feed_newIndex.iterrows():\n    start_msg = row[\'Message\']\n    words = start_msg.split()\n    check = "Vol"\n    if check in words:\n        new_msg = start_msg\n\n    check_str = "ended"\n    if "ended" in start_msg:\n        next_le = only_feed_newIndex.loc[idex+1][\'Message\']\n        if "Vol" not in next_le:\n            only_feed_newIndex.loc[idex+0.5] = new_msg\n            \nonly_feed_newIndex = only_feed_newIndex.sort_index().reset_index(drop=True)\n'

In [145]:
for idex, row in only_feed_newIndex.iterrows():
    start_msg = row['Message']
    words = start_msg.split()
    check = "Vol"
    if check in words:
        new_msg = start_msg

    check_str = "ended"
    if "ended" in start_msg:
        next_le = only_feed_newIndex.loc[idex+1]['Message']
        if "Vol" not in next_le:
            line = pd.DataFrame({"Message": new_msg}, index=[idex+0.5])
            only_feed_newIndex = only_feed_newIndex.append(line, ignore_index=False)
only_feed_newIndex = only_feed_newIndex.sort_index().reset_index(drop=True)

In [146]:
only_feed_newIndex.head(60)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
0,2021-4-20 13:4:7,1.618917e+09,639887.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
1,2021-4-20 13:4:25,1.618917e+09,177004.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
2,2021-4-20 14:5:54,1.618920e+09,625413.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
3,2021-4-20 22:0:5,1.618949e+09,125813.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
4,2021-4-20 22:0:6,1.618949e+09,644921.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
5,2021-4-20 23:40:9,1.618955e+09,644779.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
7,2021-4-21 11:23:12,1.618997e+09,630347.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
8,2021-4-21 13:26:31,1.619004e+09,861094.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
9,2021-4-21 22:0:5,1.619035e+09,342017.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN


In [147]:
only_feed_newIndex.tail(60)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
330,2021-6-18 11:34:1,1.624009e+09,154926.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
331,2021-6-18 22:0:1,1.624046e+09,144983.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
332,2021-6-18 22:0:2,1.624046e+09,670856.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
333,2021-6-18 23:34:12,1.624052e+09,689147.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
334,2021-6-19 10:0:1,1.624090e+09,144569.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
335,2021-6-19 10:0:2,1.624090e+09,646744.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
336,2021-6-19 11:32:30,1.624095e+09,145015.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
337,2021-6-19 22:0:1,1.624133e+09,143593.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
338,2021-6-19 22:0:2,1.624133e+09,690016.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
339,2021-6-19 23:33:27,1.624138e+09,640821.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN


In [148]:
for idex, row in only_feed_newIndex.iterrows():
    start_msg = row['Message']
    words = start_msg.split()
    check = "Vol"
    if check in words:
        new_msg = start_msg

    if (start_msg.endswith('started')):
        if idex == 0:
            continue
        else:
            pre_le = only_feed_newIndex.loc[idex-1]['Message']
            if "Vol" not in pre_le:
                line = pd.DataFrame({"Message": new_msg}, index=[(idex-1)+0.5])
                only_feed_newIndex = only_feed_newIndex.append(line, ignore_index=False)
                
only_feed_newIndex = only_feed_newIndex.sort_index().reset_index(drop=True)

In [149]:
only_feed_newIndex.head(60)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
0,2021-4-20 13:4:7,1.618917e+09,639887.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
2,2021-4-20 13:4:25,1.618917e+09,177004.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
3,2021-4-20 14:5:54,1.618920e+09,625413.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
4,2021-4-20 22:0:5,1.618949e+09,125813.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
5,2021-4-20 22:0:6,1.618949e+09,644921.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
6,2021-4-20 23:40:9,1.618955e+09,644779.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
8,2021-4-21 11:23:12,1.618997e+09,630347.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
9,2021-4-21 13:26:31,1.619004e+09,861094.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN


In [150]:
only_feed_newIndex.tail(60)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
334,2021-6-18 22:0:1,1.624046e+09,144983.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
335,2021-6-18 22:0:2,1.624046e+09,670856.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
336,2021-6-18 23:34:12,1.624052e+09,689147.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
337,2021-6-19 10:0:1,1.624090e+09,144569.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
338,2021-6-19 10:0:2,1.624090e+09,646744.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
339,2021-6-19 11:32:30,1.624095e+09,145015.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
340,2021-6-19 22:0:1,1.624133e+09,143593.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
341,2021-6-19 22:0:2,1.624133e+09,690016.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
342,2021-6-19 23:33:27,1.624138e+09,640821.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
343,2021-6-20 10:0:1,1.624176e+09,143837.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN


In [151]:
clean_feedDF = only_feed_newIndex[only_feed_newIndex['Message'].str.contains('PrgID', regex=False, case=False, na=False)]

In [152]:
clean_feedDF.head(60)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
1,NaN,NaN,NaN,NaN,NaN,NaN,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
4,2021-4-20 22:0:5,1.618949e+09,125813.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
10,2021-4-21 22:0:5,1.619035e+09,342017.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
13,2021-4-22 10:0:5,1.619078e+09,339214.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
16,2021-4-22 22:0:5,1.619122e+09,341318.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
19,2021-4-23 10:0:5,1.619165e+09,351891.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
22,2021-4-23 22:0:5,1.619208e+09,381896.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
25,2021-4-24 10:0:5,1.619251e+09,373106.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
28,2021-4-24 22:0:5,1.619294e+09,170238.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN


In [153]:
clean_feedDF.tail(60)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
219,2021-5-29 22:15:1,1.622319e+09,143694.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
222,2021-5-30 10:15:1,1.622363e+09,144012.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
225,2021-5-30 22:15:1,1.622406e+09,145811.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
228,2021-5-31 10:15:1,1.622449e+09,161253.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
231,2021-5-31 22:15:1,1.622492e+09,161339.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
234,2021-6-1 22:15:1,1.622579e+09,145456.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
237,2021-6-2 10:15:1,1.622622e+09,144397.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
240,2021-6-2 22:15:1,1.622665e+09,142976.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
243,2021-6-3 10:15:1,1.622708e+09,147516.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
246,2021-6-3 22:15:1,1.622751e+09,149034.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
